## Synonym of Baseline Object
Q: Why do we need to list the synonyms of Baseline Object?  
A: We expect a fuzzy matching method.

In [2]:
org_base = {'sh': 60, 'ID': 40, 'sed': 38, 'malware': 18, '1526': 13, '/etc/rc.local': 9, '1527': 9, '8192*1024 bytes': 7, 'status:0': 7, '23.236.66.13:50050': 6, 'status:2': 4, 'Permission:0700': 3, 'Permission:022': 3, 'status:1': 2, 'Sleep Duration': 2, '/proc/net/dev': 2, '/proc/stat': 2, 'uname': 1, '/proc/self/exe': 1, '"/prober"': 1, '/etc/rc.d/rc.local': 1, '/etc/init.d/boot.local': 1, '1528': 1, '1529': 1, '/dev/urandom': 1, 'NIC': 1, 'Timestamp': 1, '/sys/devices/system/cpu/online': 1}

synonyms = {
    'ID': ['process id', 'pid'],
    'NIC': ['']
}

def get_baseline_for_eval(org_base: list) -> list:
    return

ver4 dst_node set with length: `27` 

In [3]:
import pandas as pd
df = pd.read_excel(open('triplate_statistics.xlsx', 'rb'), sheet_name='ver4', index_col=0)  
org_base = set(df['dst_node'])
print(len(org_base))
org_base

27


{'"/prober"',
 '/dev/urandom',
 '/etc/init.d/boot.local',
 '/etc/rc.d/rc.local',
 '/etc/rc.local',
 '/proc/net/dev',
 '/proc/self/exe',
 '/proc/stat',
 '/sys/devices/system/cpu/online',
 '1526',
 '1527',
 '1528',
 '1529',
 '23.236.66.13:50050',
 '8192*1024 bytes',
 'ID',
 'NIC',
 'Permission:022',
 'Permission:0700',
 'Sleep Duration',
 'Timestamp',
 'sed',
 'sh',
 'status:0',
 'status:1',
 'status:2',
 'uname'}

In [4]:
base_data = {
    'dst_node':[],
    'type':[]
}
for i,row in df.iterrows():
    if row.dst_node not in base_data['dst_node']:
        base_data['dst_node'].append(row.dst_node)
        base_data['type'].append(row['type.1'])
pd.DataFrame(base_data)

,dst_node,type
0,uname,other
1,8192*1024 bytes,other
2,/proc/self/exe,file
3,"""/prober""",file
4,sh,proc
5,1526,proc
6,status:0,other
7,ID,other
8,sed,proc
9,/etc/rc.local,file


### 使用 Dofloo 的文章來測試

In [46]:
import re
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from IPython.display import display

In [18]:
report_fir_path = '../C parse report/sentence csvs/'

def get_all_filenames(dir: str='./') -> list:
    ''' traverse root directory, and list directories as dirs and files as files. Return filenames in rootdir. '''
    files = [f for f in listdir(dir) if isfile(join(dir, f))]
    files.sort()
    return files

dofloo_report_names = [f for f in get_all_filenames(report_fir_path) if f.startswith('Dofloo')]
dofloo_report_dfs = [pd.read_csv(f"{report_fir_path}{f}") for f in dofloo_report_names]
dofloo_report_names

['Dofloo-BleepingComputer.csv',
 'Dofloo-MalwareMustDie.csv',
 'Dofloo-Securityaffairs.csv',
 'Dofloo-SyscallParty.csv',
 'Dofloo-Trendmicro.csv']

In [7]:
dofloo_report_dfs[0]

,Number,Content
0,"""Sentence 1","exposed docker apis abused by ddos, cryptojack..."
1,"""Sentence 2",the dofloo (aka aesddos) malware was first det...
2,"""Sentence 3",misconfigured docker services being abused is ...
3,"""Sentence 4",allowing external access — whether intentional...
4,"""Sentence 5",query to list all available containers\nthe at...
5,"""Sentence 6",the dofloo botnet malware is subsequently depl...
6,"""Sentence 7",system information is also collected by the tr...
7,"""Sentence 8",dofloo (aesddos) malware executing remote shel...
8,"""Sentence 9",previous dofloo and docker attacks\ndofloo was...
9,"""Sentence 10",exposed docker apis were abused by other malic...


In [50]:
class ReportEvalModel:
    def __init__(self, baseline, sentences, reportname=''):
        self.baseline = [ re.sub(r"\"", '', base) for base in baseline]
        self.sentences = sentences # a df
        self.reportname = reportname
        self.match_tbl = pd.DataFrame([[0]*len(baseline)]*len(sentences), columns=baseline)
        self.match_baseline = set()
        self.match()

    def match(self):
        sentence_list = self.sentences['Content']
        for idx_sent, sent in enumerate(sentence_list):
            # print(idx_sent, sent)
            for idx_base, base in enumerate(self.baseline):
                # find baseline in a sentence
                if str(sent).find(base) != -1:
                    self.match_tbl.loc[idx_sent,base] = 1
                    self.match_baseline.add(base)
        pass
    
    def get_match_sentences(self):
        result = self.match_tbl.copy()
        result['match'] = result.sum(axis=1)
        return result
    
    def get_match_baselines(self) -> set:
        return self.match_baseline

rem = ReportEvalModel(org_base, dofloo_report_dfs[4])

In [31]:
# er.match_tbl
rem.get_match_sentences()

,/etc/rc.d/rc.local,23.236.66.13:50050,1528,1527,Timestamp,"""/prober""",ID,Sleep Duration,/etc/init.d/boot.local,status:2,...,1529,sh,NIC,/proc/net/dev,1526,sed,/etc/rc.local,Permission:022,Permission:0700,match
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,4
5,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,3
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
rem.get_match_baselines()

['/etc/rc.d/rc.local',
 '/etc/init.d/boot.local',
 'sed',
 '/etc/rc.local',
 'sh',
 '/proc/stat',
 '/proc/self/exe',
 '/proc/net/dev']

In [52]:
class FamilySet:
    def __init__(self, familyname, baseline):
        self.familyname = familyname
        self.baseline = [ re.sub(r"\"", '', base) for base in baseline]
        self.rem_lst = [] # list of ReportEvalModel under this family

    def add_rem(self, rem_to_add):
        self.rem_lst.append(rem_to_add)

    def show_result(self):
        '''print and return result table. baselinse(x) * report(y).'''
        column_names = ['report_name','ttl_match']
        column_names.extend(self.baseline)
        result_tbl = pd.DataFrame([[0]*len(column_names)]*len(self.rem_lst), columns=column_names)
        for i,rem in enumerate(self.rem_lst):
            match_baselines = rem.get_match_baselines()
            result_tbl.loc[i,'report_name'] = rem.reportname
            result_tbl.loc[i,'ttl_match'] = len(match_baselines)
            for b in match_baselines:
                result_tbl.loc[i,b] = 1
            print(rem.reportname, match_baselines)
        display(result_tbl)
        return result_tbl

fset = FamilySet('Dofloo', org_base)
for i,rdf in enumerate(dofloo_report_dfs):
    rem = ReportEvalModel(org_base, rdf, dofloo_report_names[i])
    fset.add_rem(rem)
result_tbl = fset.show_result()
# result_tbl.to_excel('FamilySet_Dofloo.xlsx', index=False)

Dofloo-BleepingComputer.csv {'sed', 'sh'}
Dofloo-MalwareMustDie.csv {'sed', 'sh'}
Dofloo-Securityaffairs.csv {'sed', '/etc/rc.local', 'sh', '/etc/rc.d/rc.local'}
Dofloo-SyscallParty.csv {'uname', '/etc/init.d/boot.local', 'sed', '/proc/net/dev', '/etc/rc.local', '/proc/self/exe', 'sh'}
Dofloo-Trendmicro.csv {'/etc/rc.d/rc.local', '/etc/init.d/boot.local', 'sed', '/proc/net/dev', '/etc/rc.local', '/proc/self/exe', 'sh', '/proc/stat'}


,report_name,ttl_match,/etc/rc.d/rc.local,23.236.66.13:50050,1528,1527,Timestamp,/prober,ID,Sleep Duration,...,/proc/self/exe,1529,sh,NIC,/proc/net/dev,1526,sed,/etc/rc.local,Permission:022,Permission:0700
0,Dofloo-BleepingComputer.csv,2,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,Dofloo-MalwareMustDie.csv,2,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,Dofloo-Securityaffairs.csv,4,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
3,Dofloo-SyscallParty.csv,7,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,1,0,0
4,Dofloo-Trendmicro.csv,8,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,1,0,0
